In [1]:
import pandas as pd
import re
import gc
import sys
#from utils import write_status
from nltk.stem.porter import PorterStemmer
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, Dropout, concatenate, Layer, InputSpec
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv('train/train.csv')
df2 = pd.read_csv('train/game_overview.csv')

In [3]:
print(df1.shape)
print(df1.isnull().sum())
df1.head()

(17494, 5)
review_id            0
title                0
year               178
user_review          0
user_suggestion      0
dtype: int64


,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1


In [4]:
print(df2.shape)
print(df2.isnull().sum())
df2.head()

(64, 5)
title        0
developer    0
publisher    0
tags         0
overview     0
dtype: int64


,title,developer,publisher,tags,overview
0,Spooky's Jump Scare Mansion,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...
1,Sakura Clicker,Winged Cloud,Winged Cloud,"['Nudity', 'Anime', 'Free to Play', 'Mature', ...",The latest entry in the Sakura series is more ...
2,WARMODE,WARTEAM,WARTEAM,"['Early Access', 'Free to Play', 'FPS', 'Multi...",Free to play shooter about the confrontation o...
3,Fractured Space,Edge Case Games Ltd.,Edge Case Games Ltd.,"['Space', 'Multiplayer', 'Free to Play', 'PvP'...",Take the helm of a gigantic capital ship and g...
4,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,"['FPS', 'Multiplayer', 'Shooter', 'Action', 'T...",Counter-Strike: Global Offensive (CS: GO) expa...


In [5]:
df = pd.merge(df1,df2,on='title')

In [6]:
print(df.shape)
df.head()

(17494, 9)


,review_id,title,year,user_review,user_suggestion,developer,publisher,tags,overview
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1,Lag Studios,Lag Studios,"['Horror', 'Free to Play', 'Cute', 'First-Pers...",Can you survive 1000 rooms of cute terror? Or ...


In [7]:
df.isnull().sum()

review_id            0
title                0
year               178
user_review          0
user_suggestion      0
developer            0
publisher            0
tags                 0
overview             0
dtype: int64

In [8]:
train_labels = df['user_suggestion']

In [9]:
df[df['user_review'].str.find('♥')>0]['user_review']


23       One of the best horror games imo. It's free, i...
34       This little gem is absolutely pants ♥♥♥♥♥♥♥♥ly...
49       This game is basically trying to tell us that ...
72       Spooky is like that crazy ex girlfriend who ca...
78       Hi to all reading this wonderful review.After ...
98       it's pretty obviously part of the seemingly-no...
105      Early Access ReviewI was all like awwww cute!!...
112      Cute terror my♥♥♥♥♥.. This thing scared the li...
116      This game is really boring.As soon as you star...
124      Early Access Reviewi think the spookiest thing...
137      Wandered through about 10 rooms without seeing...
152      This game has scared me to no end, I......I......
154      For those who are tempted to try this game out...
155      Great game its not to scary i feel its good fo...
164      At first ı thought this was just a small jump ...
167      Early Access ReviewHell yeah just came back fr...
169      First 13 minutes of gameplay (aka first 100 ro.

In [32]:
#replace url


In [29]:
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    word = word.strip('@')
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


In [10]:
def handle_emojis(col):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    col = col.replace(to_replace = '(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', value =  ' EMO_POS ', regex = True)
    #tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    col = col.replace(to_replace = '(:\s?D|:-D|x-?D|X-?D)', value = ' EMO_POS ',  regex = True)
    # Love -- <3, :*
    col = col.replace(to_replace = '(<3|:\*)', value = ' EMO_POS ', regex = True)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    col = col.replace(to_replace = '(;-?\)|;-?D|\(-?;)',value =  ' EMO_POS ', regex = True)
    # Sad -- :-(, : (, :(, ):, )-:
    col = col.replace(to_replace = '(:\s?\(|:-\(|\)\s?:|\)-:)',value =  ' EMO_NEG ', regex = True)
    # Cry -- :,(, :'(, :"(
    col = col.replace(to_replace = '(:,\(|:\'\(|:"\()', value = ' EMO_NEG ', regex = True)
    
    return col

In [11]:
#convert to lower case
df['user_review'] = df['user_review'].str.lower()
#Remove Urls
df['user_review'] = df['user_review'].str.replace('http\S+|www.\S+', '', case=False)


#remove #,@
df['user_review'] = df['user_review'].str.replace('[@#]', '')

#replace & with and
df['user_review'] = df['user_review'].str.replace('&', 'and')
#handle emojis
df['user_review'] = handle_emojis(df['user_review'])
#remove multiple spaces with single space
df['user_review'] = df['user_review'].str.replace(r'\s+', ' ')
#Remove punchutions
df['user_review'] = df['user_review'].str.replace('[\'"?!,.():;]','')
df['user_review'] = df['user_review'].replace(to_replace = r'(.)\1+',value = r'\1\1',regex = True)
df['user_review'] = df['user_review'].replace(to_replace = r'♥♥',value = r' love ',regex = True)
df['user_review'][34]

'this little gem is absolutely pants  love ly terrifying how i hear you cry surely because of the cutesy graphics it couldnt possibly be scary right wrong dead wrong ahem the soft graphics are what the game uses to lull you into a false sense of security after gaining your trust the game begins piling on the atmosphere like a dumptruck at adumptruck convention after the first 50-100 rooms or so it becomes clear that the cutesy cardboard jumpscares arent the only thing plagueing you as you journey through what quickly becomes a monstrous hellscape born out of your own nightmares the game also nods and pays homage to most genres of horror and even references some popular horror games such as five nights at freddys and resident evil the game isnt very long but if youre a coward like me then youll only be able to play it in short 200 room bursts if youre bored and looking for a sp00ky time then pick up spookys house of jump scares and prepare to spend a lot of money on new underwear becaus

In [12]:
df['user_review'][77] #♥

'aa'

In [13]:

max_features = 20000
maxlen = 100

In [14]:
tokenizer = text.Tokenizer(num_words=max_features)

In [15]:
tokenizer.fit_on_texts(df['user_review'])

In [16]:
tokenized_train = tokenizer.texts_to_sequences(df['user_review'])
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [17]:
X_train[0]

array([14514,   215,    11,     5,     9,  3498,     3,  3109,    25,
          35,   932,   603,   183,    15,    16,     1,   176, 17973,
           7,    35,  3620,    15,    36, 11159,     3,  2547,  3148,
          42,    17,    14,   264,     2,   244,    56,   162, 14515,
           3,     4,   203,  2044,     8,   230,    59,     4,   134,
        1816,   124,   192,   239,   232,     4,   298,   623,   279,
           7,  4973,    43,   792,  2375,   576,  3743,    35,  3805,
          19,    85,     3,  1012,    42,  5257,    37, 17974,  4421,
        3109,  1001,   434,   363,   169,     3,   116,    24,     8,
          23,   116,   912,  3432, 17975,   147,    12,     4,     5,
          18,     8,   104,    11,  2548,     7,  1425,     4,  1116,
        1937], dtype=int32)

In [18]:
EMBEDDING_FILE = r'/Users/Trinadh.Singaladevi/analytics vidya/Game review/twitter-sentiment-analysis-master/twitter-sentiment-analysis-master/dataset/glove.twitter.27B/glove.twitter.27B.100d.txt'
#/Users/Trinadh.Singaladevi/analytics vidya/Game review/twitter-sentiment-analysis-master/twitter-sentiment-analysis-master/dataset/glove.twitter.27B

In [19]:
import numpy as np
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [20]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size)) #embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/Users/Trinadh.Singaladevi/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [21]:
del  tokenized_train, tokenizer, word_index, embeddings_index, all_embs, nb_words
gc.collect()

100

In [22]:
batch_size = 2048
epochs = 7
embed_size = 100

In [23]:
def lstm_model(conv_layers = 2, max_dilation_rate = 3):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
    x = Dropout(0.25)(x)
    x = Conv1D(2*embed_size, kernel_size = 3)(x)
    prefilt = Conv1D(2*embed_size, kernel_size = 3)(x)
    x = prefilt
    for strides in [1, 1, 2]:
        x = Conv1D(128*2**(strides), strides = strides, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_size=3, kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x_f = LSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)  
    x_b = LSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x = concatenate([x_f, x_b])
    x = Dropout(0.5)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])

    return model

lstm_model = lstm_model()
lstm_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     2000000     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 100, 100)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D

In [24]:
weight_path="early_weights.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks = [checkpoint, early_stopping]

In [25]:
lstm_model.fit(X_train, train_labels, batch_size=batch_size, epochs=epochs, shuffle = True, validation_split=0.20, callbacks=callbacks)


Train on 13995 samples, validate on 3499 samples
Epoch 1/7
13995/13995 [==============================] - 969s 69ms/step - loss: 0.8600 - binary_accuracy: 0.5529 - val_loss: 0.8555 - val_binary_accuracy: 0.4118

Epoch 00001: val_loss improved from inf to 0.85547, saving model to early_weights.hdf5
Epoch 2/7
13995/13995 [==============================] - 1041s 74ms/step - loss: 0.6869 - binary_accuracy: 0.6087 - val_loss: 0.7473 - val_binary_accuracy: 0.4115

Epoch 00002: val_loss improved from 0.85547 to 0.74733, saving model to early_weights.hdf5
Epoch 3/7
13995/13995 [==============================] - 983s 70ms/step - loss: 0.6803 - binary_accuracy: 0.6094 - val_loss: 0.7772 - val_binary_accuracy: 0.4110

Epoch 00003: val_loss did not improve from 0.74733
Epoch 4/7
13995/13995 [==============================] - 959s 68ms/step - loss: 0.6790 - binary_accuracy: 0.6095 - val_loss: 0.7490 - val_binary_accuracy: 0.4110

Epoch 00004: val_loss did not improve from 0.74733
Epoch 5/7
10240/1

KeyboardInterrupt: 